In [6]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("match_data\ind-vs-pak-super-fours.csv")
df.head()

,Unnamed: 0,batter,baller,ball_speed,ball_length,ball_line,wicket,wide,runs,shot,commentary
0,0,Bumrah,Naseem Shah,119.0,NaN,NaN,1,0,0,NaN,"Naseem Shah to Bumrah, out Caught by Agha Salm..."
1,1,Bumrah,Naseem Shah,139.0,full,straight,0,0,4,lofted off drive,"Naseem Shah to Bumrah, FOUR, 139kph, fuller in..."
2,2,Siraj,Naseem Shah,NaN,short,outside,0,0,1,late cut,"Naseem Shah to Siraj, 1 run, short of length d..."
3,3,Kuldeep Yadav,Naseem Shah,141.0,short,straight,1,0,0,pull,"Naseem Shah to Kuldeep Yadav, out Caught by Ri..."
4,4,Kuldeep Yadav,Naseem Shah,NaN,length,outside,0,0,0,late cut,"Naseem Shah to Kuldeep Yadav, no run, B0$, len..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   305 non-null    int64  
 1   batter       305 non-null    object 
 2   baller       305 non-null    object 
 3   ball_speed   113 non-null    float64
 4   ball_length  258 non-null    object 
 5   ball_line    266 non-null    object 
 6   wicket       305 non-null    int64  
 7   wide         305 non-null    int64  
 8   runs         305 non-null    int64  
 9   shot         258 non-null    object 
 10  commentary   305 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 26.3+ KB


In [9]:
df.columns

Index(['Unnamed: 0', 'batter', 'baller', 'ball_speed', 'ball_length',
       'ball_line', 'wicket', 'wide', 'runs', 'shot', 'commentary'],
      dtype='object')

In [10]:
columns_to_drop = ['Unnamed: 0', 'batter', 'baller', 'ball_speed', 'wicket', 'wide', 'runs']
df.drop(columns=columns_to_drop, inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257 entries, 1 to 303
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ball_length  257 non-null    object
 1   ball_line    257 non-null    object
 2   shot         257 non-null    object
 3   commentary   257 non-null    object
dtypes: object(4)
memory usage: 10.0+ KB


In [11]:
ball_length=df["ball_length"]
ball_line=df["ball_line"]
shot=df["shot"]

labels_ball_length=[]
for j in ball_length:
    labels_ball_length.append(j)

labels_ball_line=[]
for j in ball_line:
    labels_ball_line.append(j)

labels_shot=[]
for j in shot:
    labels_shot.append(j)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

text=[]
for i in df["commentary"]:
    text.append(i)
    # print(i)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=500)

import joblib

tfidf_vectorizer_filename = 'tfidf_vectorizer.pkl'
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_filename)

# Compute TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(text)

In [13]:
# ball_line

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels_ball_line, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=99, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)



import joblib

# Save the trained Random Forest classifier to a file
model_filename = 'ball_line_model.pkl'
joblib.dump(rf_classifier, model_filename)

Accuracy: 0.7307692307692307
              precision    recall  f1-score   support

         leg       0.50      0.50      0.50         4
     outside       0.77      0.83      0.80        24
    straight       0.73      0.67      0.70        24

    accuracy                           0.73        52
   macro avg       0.67      0.67      0.67        52
weighted avg       0.73      0.73      0.73        52



['ball_line_model.pkl']

In [14]:
# ball_length

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels_ball_length, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42,max_depth=14)

# Train the classifier
rf_classifier.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)



import joblib

# Save the trained Random Forest classifier to a file
model_filename = 'ball_length_model.pkl'
joblib.dump(rf_classifier, model_filename)

Accuracy: 0.7884615384615384
              precision    recall  f1-score   support

        full       0.80      0.92      0.86        26
      length       0.80      0.50      0.62        16
       short       0.75      0.90      0.82        10

    accuracy                           0.79        52
   macro avg       0.78      0.77      0.76        52
weighted avg       0.79      0.79      0.78        52



['ball_length_model.pkl']

In [15]:
# shot

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels_shot, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=120, random_state=42,max_depth=12)

# Train the classifier
rf_classifier.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)




import joblib

# Save the trained Random Forest classifier to a file
model_filename = 'shot_model.pkl'
joblib.dump(rf_classifier, model_filename)

Accuracy: 0.5961538461538461
                precision    recall  f1-score   support

backfoot punch       0.00      0.00      0.00         1
   cover drive       0.71      0.71      0.71         7
           cut       0.45      0.62      0.53         8
        defend       0.42      0.83      0.56         6
         flick       0.64      1.00      0.78         7
      late cut       0.75      1.00      0.86         3
         leave       0.00      0.00      0.00         2
          miss       0.00      0.00      0.00         3
     off drive       0.50      0.25      0.33         4
      on drive       1.00      0.67      0.80         6
          pull       1.00      0.33      0.50         3
straight drive       0.00      0.00      0.00         2

      accuracy                           0.60        52
     macro avg       0.46      0.45      0.42        52
  weighted avg       0.55      0.60      0.54        52



c:\Users\gupta\Desktop\ICC player dataset\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\gupta\Desktop\ICC player dataset\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\gupta\Desktop\ICC player dataset\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

['shot_model.pkl']

In [27]:
import pandas as pd
test_df=pd.read_csv("match_data\ind-vs-pak.csv")
test_df.head()

,Unnamed: 0,batter,baller,ball_speed,ball_length,ball_swing,wicket,wide,runs,shot,commentary
0,0,Bumrah,Naseem Shah,Slower ball at 119,NaN,NaN,1,0,0,NaN,"Naseem Shah to Bumrah, out Caught by Agha Salm..."
1,1,Bumrah,Naseem Shah,139,fuller,NaN,0,0,4,NaN,"Naseem Shah to Bumrah, FOUR, 139kph, fuller in..."
2,2,Siraj,Naseem Shah,NaN,short,NaN,0,0,1,NaN,"Naseem Shah to Siraj, 1 run, short of length d..."
3,3,Kuldeep Yadav,Naseem Shah,141,short,outside,1,0,0,NaN,"Naseem Shah to Kuldeep Yadav, out Caught by Ri..."
4,4,Kuldeep Yadav,Naseem Shah,NaN,length,NaN,0,0,0,NaN,"Naseem Shah to Kuldeep Yadav, no run, B0$, len..."


In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   305 non-null    int64  
 1   batter       305 non-null    object 
 2   baller       305 non-null    object 
 3   ball_speed   113 non-null    float64
 4   ball_length  258 non-null    object 
 5   ball_line    266 non-null    object 
 6   wicket       305 non-null    int64  
 7   wide         305 non-null    int64  
 8   runs         305 non-null    int64  
 9   shot         258 non-null    object 
 10  commentary   305 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 26.3+ KB


In [28]:
x=[]
length=test_df.commentary
for j in length:
    x.append(j)

x = tfidf_vectorizer.fit_transform(x)

# Load the saved model
ball_length_model = joblib.load("models/ball_length_model.pkl")
ball_line_model = joblib.load("models/ball_line_model.pkl")
shot_model = joblib.load("models/shot_model.pkl")

# Use the loaded model to make predictions
predictions_a = ball_length_model.predict(x)
predictions_b = ball_line_model.predict(x)
predictions_c = shot_model.predict(x)

test_df["ball_length"]=predictions_a
test_df["ball_swing"]=predictions_b
test_df["shot"]=predictions_c

test_df.to_csv("match_data\ind-vs-pak.csv", index=False)